# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> #### Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Módulo

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

In [4]:
#include <stdio.h>

int main(){
    float valor = 200;
    float juros = 0.01;
    int n_de_parcelas = 5;
    for(int i=0;i<n_de_parcelas;i++){
        printf("parcela %d: %.2f \n",i+1,valor);
        valor += valor*juros;
    }
    return 0;
}

parcela 1: 200.00 
parcela 2: 202.00 
parcela 3: 204.02 
parcela 4: 206.06 
parcela 5: 208.12 


## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de uma única parcela X do empréstimo. Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa principal. A função deve apenas calcular uma única parcela, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [16]:
#include <stdio.h>

float proximaParcela(float valor,float juros, int x){
    float parcela = valor ;
    for(int i=0;i<x;i++){
        parcela += parcela*juros;
    }
    return parcela ;
}

int main(){
    float valor = 200;
    float juros = 0.01;
    int n_de_parcelas = 5;
    for(int i=0;i<n_de_parcelas;i++){
        printf("parcela %d : %.2f \n", i+1,proximaParcela(valor,juros,i));
    }
    return 0;
}

parcela 1 : 200.00 
parcela 2 : 202.00 
parcela 3 : 204.02 
parcela 4 : 206.06 
parcela 5 : 208.12 


## Exercício Parte 3 - Mantendo o Estado

Modifique a função (módulo) `proximaParcela` acima de modo que ela tenha uma memória da situação em que estão as coisas, sem que o `main` precise ficar informando novamente coisas que já informou no início. Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência dos módulos com o programa principal.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente.

In [20]:
#include <stdio.h>

float proximaParcela(float valor, float juros,int n_de_parcelas){
    static int atual = 1;
    if(atual == 1){
        atual++;
        return valor;
    }
    else if(atual <= n_de_parcelas ){
        valor += valor*juros;
    }
    atual ++;
    return valor;
}

int main(){
    float valor = 200;
    float juros = 0.01;
    int n_de_parcelas = 5;
    int atual;
    for(int i=0; i<n_de_parcelas;i++){
        valor = proximaParcela(valor,juros,n_de_parcelas);
        printf("parcela %d: %.2f \n", i+1,valor);
    }
    return 0;
}

parcela 1: 200.00 
parcela 2: 202.00 
parcela 3: 204.02 
parcela 4: 206.06 
parcela 5: 208.12 


## Exercício Parte 4 - Minimizando os Parâmetros

Retomando a solução da `Parte 2`, em que a função `main` deve ter usado vários parâmetros para a comunicação com a função `proximaParcela`,  modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar o estado completo do empréstimo.

In [1]:
#include <stdio.h>

typedef struct{
    float valor;
    float juros;
    int n_de_parcelas;
    int atual;
} Emprestimo;

float proximaParcela(Emprestimo *emprestimo){
    float parcela = emprestimo->valor;
    if(emprestimo->atual < emprestimo->n_de_parcelas){
        emprestimo->valor += emprestimo->valor*emprestimo->juros;
    }
    else{
        emprestimo->valor = -1;
    }
    emprestimo->atual ++;
    return parcela;
}

int main(){
    Emprestimo emprestimo1;
    emprestimo1.valor = 200;
    emprestimo1.juros = 0.01;
    emprestimo1.n_de_parcelas = 5;
    emprestimo1.atual = 1;
    int i = 1;
    while(emprestimo1.valor != -1){
        printf("parcela %d: %.2f \n", i, emprestimo1.valor);
        proximaParcela(&emprestimo1);
    }
    return 0;
}


parcela 1: 200.00 
parcela 1: 202.00 
parcela 1: 204.02 
parcela 1: 206.06 
parcela 1: 208.12 


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada empréstimo separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, a função `proximaParcela` ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que, cada novo empréstimo só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo.

In [5]:
#include <stdio.h>

typedef struct{
    float valor;
    float juros;
    int n_de_parcelas;
    int atual;
} Emprestimo;

float proximaParcela(Emprestimo *emprestimo){
    float parcela = emprestimo->valor;
    if(emprestimo->atual < emprestimo->n_de_parcelas){
        emprestimo->valor += emprestimo->valor*emprestimo->juros;
    }
    else{
        emprestimo->valor = -1;
    }
    emprestimo->atual ++;
    return parcela;
}

void inicializa_emprestimo(Emprestimo *emprestimo, float valor, float juros, int N){
    emprestimo->valor = valor;
    emprestimo->juros = juros;
    emprestimo->n_de_parcelas = N;
    emprestimo->atual = 1;
}

int main(){
    Emprestimo emprestimo1;
    inicializa_emprestimo(&emprestimo1,200,0.01,5);
    Emprestimo emprestimo2;
    inicializa_emprestimo(&emprestimo2,500,0.02,7);
    int fim = 0;
    int i1 = 1;
    int i2 = 1;
    while(fim != 1){
        if(emprestimo1.valor == -1 && emprestimo2.valor == -1){
            fim = 1;
        }
        else{
            if(emprestimo1.valor != -1){
                printf("emprestimo 1: parcela %d eh %.2f\n", i1, emprestimo1.valor);
                proximaParcela(&emprestimo1);
                i1++;
            }
            if(emprestimo2.valor != -1){
                printf("emprestimo 2: parcela %d eh %.2f\n", i2, emprestimo2.valor);
                proximaParcela(&emprestimo2);
                i2++;
            }
        }
    }
    return 0;
}

emprestimo 1: parcela 1 eh 200.00
emprestimo 2: parcela 1 eh 500.00
emprestimo 1: parcela 2 eh 202.00
emprestimo 2: parcela 2 eh 510.00
emprestimo 1: parcela 3 eh 204.02
emprestimo 2: parcela 3 eh 520.20
emprestimo 1: parcela 4 eh 206.06
emprestimo 2: parcela 4 eh 530.60
emprestimo 1: parcela 5 eh 208.12
emprestimo 2: parcela 5 eh 541.22
emprestimo 2: parcela 6 eh 552.04
emprestimo 2: parcela 7 eh 563.08
